In [ ]:
#!/usr/bin/env python3

import sys
sys.path.insert(4,'/home/oliver/computer_vision/classification/SlowFast') # Path to the SlowFast modules

"""Wrapper to train and test a video classification model."""
from slowfast.config.defaults import assert_and_infer_cfg
from slowfast.utils.misc import launch_job
from slowfast.utils.parser import load_config, parse_args

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ['CUDA_LAUNCH_BLOCKING']="1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

# use tools module inside slowfast (since some built-in modules have the same name 'tools')
import importlib.util
spec = importlib.util.spec_from_file_location('tools', '/home/oliver/computer_vision/classification/SlowFast/tools/__init__.py')
custom_module = importlib.util.module_from_spec(spec)
sys.modules['tools'] = custom_module
spec.loader.exec_module(custom_module)

from tools.demo_net import demo
from tools.test_net import test
from tools.train_net import train
from tools.visualization import visualize

/home/oliver/computer_vision/classification/env_SlowFast/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/oliver/computer_vision/classification/env_SlowFast/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [ ]:
import argparse

parser = argparse.ArgumentParser(
        description="Provide SlowFast video training and testing pipeline."
    )
parser.add_argument(
    "--shard_id",
    help="The shard id of current node, Starts from 0 to num_shards - 1",
    default=0,
    type=int,
)
parser.add_argument(
    "--num_shards",
    help="Number of shards using by the job",
    default=1,
    type=int,
)
parser.add_argument(
    "--init_method",
    help="Initialization method, includes TCP or shared file-system",
    default="tcp://localhost:9999",
    type=str,
)
parser.add_argument(
    "--cfg",
    dest="cfg_files",
    help="Path to the config files",
    default=["configs/Kinetics/SLOWFAST_4x16_R50.yaml"],
    nargs="+",
)
parser.add_argument(
    "--opts",
    help="See slowfast/config/defaults.py for all options",
    default=None,
    nargs=argparse.REMAINDER,
)

path_to_config='./SLOWFAST_32x2_R101_50_50.yaml'

args = parser.parse_args(args=['--cfg',path_to_config])

In [3]:
cfg = load_config(args, path_to_config)
cfg = assert_and_infer_cfg(cfg)

# cfg.NUM_GPUS = 1

# Perform training.
if cfg.TRAIN.ENABLE:
    launch_job(cfg=cfg, init_method=args.init_method, func=train)

# Perform multi-clip testing.
if cfg.TEST.ENABLE:
    if cfg.TEST.NUM_ENSEMBLE_VIEWS == -1:
        num_view_list = [1, 3, 5, 7, 10]
        for num_view in num_view_list:
            cfg.TEST.NUM_ENSEMBLE_VIEWS = num_view
            launch_job(cfg=cfg, init_method=args.init_method, func=test)
    else:
        launch_job(cfg=cfg, init_method=args.init_method, func=test)

# Perform model visualization.
if cfg.TENSORBOARD.ENABLE and (
    cfg.TENSORBOARD.MODEL_VIS.ENABLE or cfg.TENSORBOARD.WRONG_PRED_VIS.ENABLE
):
    launch_job(cfg=cfg, init_method=args.init_method, func=visualize)

# Run demo.
# if cfg.DEMO.ENABLE:
#     demo(cfg)




[12/11 10:03:30][INFO] train_net.py:  534: Train with config:


[12/11 10:03:30][INFO] train_net.py:  535: CfgNode({'CONTRASTIVE': CfgNode({'T': 0.07, 'DIM': 128, 'LENGTH': 239975, 'QUEUE_LEN': 65536, 'MOMENTUM': 0.5, 'MOMENTUM_ANNEALING': False, 'TYPE': 'mem', 'INTERP_MEMORY': False, 'MEM_TYPE': '1d', 'NUM_CLASSES_DOWNSTREAM': 400, 'NUM_MLP_LAYERS': 1, 'MLP_DIM': 2048, 'BN_MLP': False, 'BN_SYNC_MLP': False, 'LOCAL_SHUFFLE_BN': True, 'MOCO_MULTI_VIEW_QUEUE': False, 'DELTA_CLIPS_MIN': -inf, 'DELTA_CLIPS_MAX': inf, 'PREDICTOR_DEPTHS': [], 'SEQUENTIAL': False, 'SIMCLR_DIST_ON': True, 'SWAV_QEUE_LEN': 0, 'KNN_ON': True}), 'BN': CfgNode({'USE_PRECISE_STATS': False, 'NUM_BATCHES_PRECISE': 200, 'WEIGHT_DECAY': 0.0, 'NORM_TYPE': 'batchnorm', 'NUM_SPLITS': 1, 'NUM_SYNC_DEVICES': 1, 'GLOBAL_SYNC': False}), 'TRAIN': CfgNode({'ENABLE': True, 'KILL_LOSS_EXPLOSION_FACTOR': 0.0, 'DATASET': 'ava', 'BATCH_SIZE': 8, 'EVAL_PERIOD': 1, 'CHECKPOINT_PERIOD': 1, 'AUTO_RESUME': False, 'CHECKPOINT_FILE_PATH': '/home/pigs/SlowFast_models/test_data_prep/test_data_run/SLOWFAS

/home/oliver/computer_vision/classification/SlowFast/tools/train_net.py:546: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=cfg.TRAIN.MIXED_PRECISION)


[12/11 10:03:33][INFO] train_net.py:  627: Start epoch: 1


/home/oliver/computer_vision/classification/SlowFast/tools/train_net.py:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=cfg.TRAIN.MIXED_PRECISION):


[12/11 10:03:39][INFO] logging.py:   98: json_stats: {"_type": "train_iter", "cur_epoch": "1/10", "cur_iter": "10", "dt": 0.29867, "dt_data": 0.00028, "dt_net": 0.29839, "eta": "0:00:15", "loss": 0.68160, "lr": 0.00512, "mode": "train"}
[12/11 10:03:43][INFO] logging.py:   98: json_stats: {"_type": "train_iter", "cur_epoch": "1/10", "cur_iter": "20", "dt": 0.29850, "dt_data": 0.00020, "dt_net": 0.29830, "eta": "0:00:12", "loss": 0.68495, "lr": 0.01067, "mode": "train"}
[12/11 10:03:48][INFO] logging.py:   98: json_stats: {"_type": "train_iter", "cur_epoch": "1/10", "cur_iter": "30", "dt": 0.29823, "dt_data": 0.00028, "dt_net": 0.29796, "eta": "0:00:09", "loss": 0.68455, "lr": 0.01622, "mode": "train"}
[12/11 10:03:53][INFO] logging.py:   98: json_stats: {"_type": "train_iter", "cur_epoch": "1/10", "cur_iter": "40", "dt": 0.29856, "dt_data": 0.00027, "dt_net": 0.29829, "eta": "0:00:06", "loss": 0.68350, "lr": 0.02176, "mode": "train"}
[12/11 10:03:58][INFO] logging.py:   98: json_stats:

KeyboardInterrupt: 